In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("Opportunity Details.xlsx", engine="openpyxl")
df.head()

,Index,-SUM([Annl Agg Savings - treated]),Customer,Super Payer Short,Payer Short,Solution,LOB,Product Type,Policy Bundle Indicator,Topic,...,Dp Desc,DP Age in Years,Disposition,Disposition Date,Cpw Reason,Decision Status,Decision Date,Model Recommendation,Annl Edits,Annl Agg Savings
0,Grand Total,Total,Total,Total,Total,Total,Total,Total,Total,Total,...,Total,Total,Total,Total,Total,Total,Total,Total,2.754112e+07,1.382905e+09
1,1,"-102,071,148.314997",Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) AMA, Implementations-Outpatient",Duplicate Claim Logic for Claims Submitted on ...,...,Deny duplicate claim lines using the 11 basic ...,12,Invalid,Jul-2025,"[""Mutually Exclusive DP (e.g. CMS vs CMS + Cot...",Reject,Oct-2018,More Likely,2.720907e+05,1.020711e+08
2,2,"-96,052,825.9297647",Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Services for the Outpatient Hospital,...,Deny packaged HCPCS codes (Status indicator N)...,12,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Less Likely,5.665652e+05,9.605283e+07
3,3,"-70,795,535.425743",Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,"(Non-D&B) AMA, Implementations-Professional",Duplicate Claims From Any Provider ID Under Sa...,...,Deny duplicate claim lines using the nine basi...,12,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",Reject,Aug-2018,More Likely,8.849959e+05,7.079554e+07
4,4,"-53,592,443.9212019",Highmark,HIGAL,HIGPA,PPM,Commercial,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Services for the Outpatient Hospital,...,Deny packaged HCPCS codes (Status indicator N)...,12,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Less Likely,3.716954e+05,5.359244e+07


In [2]:
# STEP 2: remove the "Grand Total" row, clean columns, and join PolicyCollection + PolicyGroup from mapping file

# 1) Drop the total row (keep header as-is, start data from original Index 1)
df = df.iloc[1:].reset_index(drop=True)

# 2) Drop unwanted columns
df = df.drop(columns=["Index", "-SUM([Annl Agg Savings - treated])"], errors="ignore")

# 3) Join PolicyCollection + PolicyGroup from Policy Collection Mapping (by DP Key)
df_map = pd.read_excel(
    "Policy Collection Mapping.xlsx",
    engine="openpyxl",
    usecols=["DPKey", "PolicyCollection", "PolicyGroup"]
)

df_map["DPKey"] = pd.to_numeric(df_map["DPKey"], errors="coerce")
df["Dp Key"] = pd.to_numeric(df["Dp Key"], errors="coerce")

df = (
    df.merge(df_map.drop_duplicates("DPKey"), how="left", left_on="Dp Key", right_on="DPKey")
      .drop(columns=["DPKey"])
      .rename(columns={"PolicyCollection": "Policy Collection", "PolicyGroup": "Policy Group"})
)


In [3]:
# quick view to confirm
print(df.shape)
print(df.columns.tolist())
df.head(5)

(14142, 21)
['Customer', 'Super Payer Short', 'Payer Short', 'Solution', 'LOB', 'Product Type', 'Policy Bundle Indicator', 'Topic', 'Dp Key', 'Dp Desc', 'DP Age in Years', 'Disposition', 'Disposition Date  ', 'Cpw Reason', 'Decision Status', 'Decision Date', 'Model Recommendation', 'Annl Edits', 'Annl Agg Savings', 'Policy Collection', 'Policy Group']


,Customer,Super Payer Short,Payer Short,Solution,LOB,Product Type,Policy Bundle Indicator,Topic,Dp Key,Dp Desc,...,Disposition,Disposition Date,Cpw Reason,Decision Status,Decision Date,Model Recommendation,Annl Edits,Annl Agg Savings,Policy Collection,Policy Group
0,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) AMA, Implementations-Outpatient",Duplicate Claim Logic for Claims Submitted on ...,3324,Deny duplicate claim lines using the 11 basic ...,...,Invalid,Jul-2025,"[""Mutually Exclusive DP (e.g. CMS vs CMS + Cot...",Reject,Oct-2018,More Likely,272090.742312,1.020711e+08,NaN,NaN
1,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Services for the Outpatient Hospital,11369,Deny packaged HCPCS codes (Status indicator N)...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Less Likely,566565.163348,9.605283e+07,NaN,NaN
2,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,"(Non-D&B) AMA, Implementations-Professional",Duplicate Claims From Any Provider ID Under Sa...,71,Deny duplicate claim lines using the nine basi...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",Reject,Aug-2018,More Likely,884995.874770,7.079554e+07,NaN,NaN
3,Highmark,HIGAL,HIGPA,PPM,Commercial,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Services for the Outpatient Hospital,11369,Deny packaged HCPCS codes (Status indicator N)...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Less Likely,371695.429337,5.359244e+07,NaN,NaN
4,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Revenue Codes for the Outpatient Hosp...,3518,Deny packaged revenue codes when billed withou...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",NaN,Never Presented,Less Likely,190631.752085,5.266594e+07,NaN,NaN


In [4]:
# STEP 3: Format Annl Edits + Annl Agg Savings as whole numbers and display

# Annl Edits -> whole number
if "Annl Edits" in df.columns:
    df["Annl Edits"] = pd.to_numeric(df["Annl Edits"], errors="coerce").round(0).astype("Int64")

# Annl Agg Savings -> whole number (no decimals)
if "Annl Agg Savings" in df.columns:
    df["Annl Agg Savings"] = pd.to_numeric(df["Annl Agg Savings"], errors="coerce").round(0).astype("Int64")

# Display formatting (no scientific notation, no decimals)
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

df.head(10)

,Customer,Super Payer Short,Payer Short,Solution,LOB,Product Type,Policy Bundle Indicator,Topic,Dp Key,Dp Desc,...,Disposition,Disposition Date,Cpw Reason,Decision Status,Decision Date,Model Recommendation,Annl Edits,Annl Agg Savings,Policy Collection,Policy Group
0,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) AMA, Implementations-Outpatient",Duplicate Claim Logic for Claims Submitted on ...,3324,Deny duplicate claim lines using the 11 basic ...,...,Invalid,Jul-2025,"[""Mutually Exclusive DP (e.g. CMS vs CMS + Cot...",Reject,Oct-2018,More Likely,272091,102071148,NaN,NaN
1,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Services for the Outpatient Hospital,11369,Deny packaged HCPCS codes (Status indicator N)...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Less Likely,566565,96052826,NaN,NaN
2,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,"(Non-D&B) AMA, Implementations-Professional",Duplicate Claims From Any Provider ID Under Sa...,71,Deny duplicate claim lines using the nine basi...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",Reject,Aug-2018,More Likely,884996,70795535,NaN,NaN
3,Highmark,HIGAL,HIGPA,PPM,Commercial,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Services for the Outpatient Hospital,11369,Deny packaged HCPCS codes (Status indicator N)...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Less Likely,371695,53592444,NaN,NaN
4,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Revenue Codes for the Outpatient Hosp...,3518,Deny packaged revenue codes when billed withou...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",NaN,Never Presented,Less Likely,190632,52665941,NaN,NaN
5,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,"(Non-D&B) CMS, Implementations-Outpatient, Lab...",Packaged Services for the Outpatient Hospital,9065,Deny conditionally packaged laboratory service...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Less Likely,528008,24935085,NaN,NaN
6,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,(Non-D&B) Waste & Abuse,DME Quality of Care,4651,Deny any code billed by a DME provider when th...,...,Invalid,Jul-2025,"[""Primary Reference Not Accepted by the Client""]",No Decision,Jan-2021,Less Likely,53383,23041309,NaN,NaN
7,Highmark,HIGAL,HIGPA,PPM,Commercial,ICM,(Non-D&B) Waste & Abuse,DME Quality of Care,4651,Deny any code billed by a DME provider when th...,...,Invalid,Jul-2025,"[""Primary Reference Not Accepted by the Client""]",Reject,Jan-2021,Less Likely,74833,22508408,NaN,NaN
8,Highmark,HIGAL,HIGAC,PPM,Commercial,ICMO,"(Non-D&B) CMS, Implementations-Outpatient",Packaged Services for the Outpatient Hospital,11369,Deny packaged HCPCS codes (Status indicator N)...,...,Invalid,Jul-2025,"[""Claims System Limitation (e.g. Client does n...",No Decision,Apr-2020,Likely,129866,21656720,NaN,NaN
9,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,"(Non-D&B) CMS, (D&B) Drug Wastage",Drug Wastage Modifiers JW and JZ,17227,Deny a drug supplied in only single-dose formu...,...,Invalid,Aug-2025,"[""Inappropriate for Line of Business""]",Reject,Oct-2024,Not Recommended,28278,20015094,Drug wastage,Drug Wastage


In [5]:
# STEP 4: Define filter dropdown options + apply filters + create pivot (based on selections)

FILTER_COLS = ["Recently presented", "LOB", "Decision Status", "Decision Date"]

# dropdown options (unique values for each filter column)
filter_options = {
    c: sorted(df[c].dropna().astype(str).str.strip().loc[lambda s: s.ne("")].unique().tolist())
    for c in FILTER_COLS if c in df.columns
}

# choose filters here (single value OR list of values). Use "All" or None to skip a filter.
selected_filters = {
    "LOB": ["Medicare", "Commercial"],          # example multi-select
    "Decision Status": ["Reject", "No Decision", "Suppress"],
    "Decision Date": "All"
}


In [6]:
# apply filters on base df
df_f = df.copy()
for c, v in selected_filters.items():
    if c not in df_f.columns or v is None or (isinstance(v, str) and v.strip().lower() == "all"):
        continue
    s = df_f[c].astype(str).str.strip()
    if isinstance(v, (list, tuple, set)):
        df_f = df_f[s.isin([str(x).strip() for x in v])]
    else:
        df_f = df_f[s == str(v).strip()]

# create pivoted df from filtered data
df_pivot = (
    df_f.groupby(["Topic", "Policy Collection", "Dp Key", "Payer Short"], as_index=False)[["Annl Edits", "Annl Agg Savings"]]
        .sum()
        .rename(columns={"Annl Edits": "Sum of Annl Edits", "Annl Agg Savings": "Sum of Annl Agg Savings"})
)

df_pivot.head(10)


,Topic,Policy Collection,Dp Key,Payer Short,Sum of Annl Edits,Sum of Annl Agg Savings
0,Ado-Trastuzumab Emtansine (J9354),Laboratory,8489,HIGAC,2,4123
1,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",Dual diagnosis,11674,HIGAC,112,100483
2,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",Dual diagnosis,11674,HIGDE,52,36822
3,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",Dual diagnosis,11674,HIGPA,439,372575
4,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",Dual diagnosis,11674,HIGWV,49,71798
5,Aripiprazole (Abilify Maintena) (J0401),All indications,11726,HIGAC,3,0
6,Aripiprazole (Abilify Maintena) (J0401),All indications,11726,HIGNY,6,8748
7,Aripiprazole (Abilify Maintena) (J0401),Frequency,11728,HIGNY,3,2297
8,Atezolizumab (J9022),All indications,10057,HIGAC,5,67966
9,Atezolizumab (J9022),All indications,10057,HIGDE,4,20053


In [7]:

# ---------- Build a payer-agnostic pivot (Topic + DP Key) ----------
edits_col  = "Sum of Annl Edits" if "Sum of Annl Edits" in df_pivot.columns else "Annl Edits"
sav_col    = "Sum of Annl Agg Savings" if "Sum of Annl Agg Savings" in df_pivot.columns else "Annl Agg Savings"

df_piv2 = (
    df_pivot.groupby(["Topic", "Dp Key"], as_index=False)[[edits_col, sav_col]]
              .sum()
              .rename(columns={edits_col: "Annl Edits", sav_col: "Annl Agg Savings"})
)

# ---------- Topic_Dp_Count (drives <=10 vs >10 logic) ----------
tdc = pd.read_excel("Topic_Dp_Count.xlsx", engine="openpyxl")
tdc.columns = [c.strip() for c in tdc.columns]  # safety
tdc = tdc.rename(columns={"Dp key": "Dp Key"})  # matches your screenshot
tdc["Dp Key"] = pd.to_numeric(tdc["Dp Key"], errors="coerce")
tdc["Topic_Dp_Count"] = pd.to_numeric(tdc["Topic_Dp_Count"], errors="coerce")

topic_counts = tdc.groupby("Topic", as_index=False)["Topic_Dp_Count"].max()
small_topics = set(topic_counts.loc[topic_counts["Topic_Dp_Count"] <= 10, "Topic"])
large_topics = set(topic_counts.loc[topic_counts["Topic_Dp_Count"] > 10, "Topic"])

# ---------- Decision Date lookup for comments (from filtered base df_f) ----------
def _pick_decision_date(s: pd.Series) -> str:
    vals = s.dropna().astype(str).str.strip()
    vals = vals[vals.ne("")]
    if vals.empty:
        return "Never presented"
    if (vals.str.lower().str.contains("never")).any():
        return "Never presented"
    dt = pd.to_datetime(vals, format="%b-%Y", errors="coerce")
    if dt.notna().any():
        return dt.max().strftime("%b-%Y")
    return vals.iloc[0]

dec_map = (
    df_f.assign(**{"Dp Key": pd.to_numeric(df_f["Dp Key"], errors="coerce")})
       .groupby(["Topic", "Dp Key"], as_index=False)["Decision Date"]
       .agg(_pick_decision_date)
       .rename(columns={"Decision Date": "Comments"})
)


In [8]:
# ---- Building df_to_present 

# 0) Topic+DP pivot (payer-agnostic) from df_pivot
edits_col = "Sum of Annl Edits" if "Sum of Annl Edits" in df_pivot.columns else "Annl Edits"
sav_col   = "Sum of Annl Agg Savings" if "Sum of Annl Agg Savings" in df_pivot.columns else "Annl Agg Savings"

df_piv2 = (
    df_pivot.groupby(["Topic", "Dp Key"], as_index=False)[[edits_col, sav_col]]
              .sum()
              .rename(columns={edits_col: "Annl Edits", sav_col: "Annl Agg Savings"})
)
df_piv2["Annl Edits"] = pd.to_numeric(df_piv2["Annl Edits"], errors="coerce").fillna(0)
df_piv2["Annl Agg Savings"] = pd.to_numeric(df_piv2["Annl Agg Savings"], errors="coerce").fillna(0)

# 1) Read Topic_Dp_Count
tdc = pd.read_excel("Topic_Dp_Count.xlsx", engine="openpyxl")
tdc.columns = [c.strip() for c in tdc.columns]
tdc = tdc.rename(columns={"Dp key": "Dp Key"})
tdc["Dp Key"] = pd.to_numeric(tdc["Dp Key"], errors="coerce")
tdc["Topic_Dp_Count"] = pd.to_numeric(tdc["Topic_Dp_Count"], errors="coerce")

topic_cnt = tdc.groupby("Topic", as_index=False)["Topic_Dp_Count"].max()
topic_cnt_map = dict(zip(topic_cnt["Topic"], topic_cnt["Topic_Dp_Count"]))

# 2) Decide which <=10 topics qualify (must have >=1 active DP with edits OR savings > 0)
active_topic = set(
    df_piv2.loc[(df_piv2["Annl Edits"] > 0) | (df_piv2["Annl Agg Savings"] > 0), "Topic"].unique()
)
small_topics_ok = set(topic_cnt.loc[(topic_cnt["Topic_Dp_Count"] <= 10) & (topic_cnt["Topic"].isin(active_topic)), "Topic"])
large_topics = set(topic_cnt.loc[topic_cnt["Topic_Dp_Count"] > 10, "Topic"])

# 3) Latest Decision Date + matching Decision Status per (Topic, Dp Key) from df_f
df_f2 = df_f.copy()
df_f2["Dp Key"] = pd.to_numeric(df_f2["Dp Key"], errors="coerce")
df_f2["_dd_dt"] = pd.to_datetime(df_f2["Decision Date"].astype(str).str.strip(), format="%b-%Y", errors="coerce")

latest_idx = (
    df_f2.sort_values(["Topic", "Dp Key", "_dd_dt"])
         .groupby(["Topic", "Dp Key"], as_index=False)
         .tail(1)
         .index
)

dec_latest = df_f2.loc[latest_idx, ["Topic", "Dp Key", "Decision Status", "_dd_dt"]].copy()
dec_latest = dec_latest.rename(columns={"_dd_dt": "DecisionDate_dt"})
dec_latest["DecisionYear"] = dec_latest["DecisionDate_dt"].dt.year

def _comment_from_status(status, year, dt):
    s = "" if pd.isna(status) else str(status).strip()
    if pd.isna(dt):
        return "Never presented"
    if s.lower() == "no decision":
        return f"No Decision in ({int(year)})" if pd.notna(year) else "No Decision"
    if s.lower() == "reject":
        return f"Previously Rejected in ({int(year)})" if pd.notna(year) else "Previously Rejected"
    if s.lower() == "suppress":
        return f"Previously Suppressed in ({int(year)})" if pd.notna(year) else "Previously Suppressed"
    # else: keep date as the comment
    return dt.strftime("%b-%Y")

dec_latest["Comments"] = dec_latest.apply(
    lambda r: _comment_from_status(r["Decision Status"], r["DecisionYear"], r["DecisionDate_dt"]),
    axis=1
)

# 4A) <=10 topics (ONLY those qualifying): include ALL DP keys -> "To complete topic"
df_small = (
    tdc.loc[tdc["Topic"].isin(small_topics_ok), ["Topic", "Dp Key"]]
       .dropna(subset=["Dp Key"])
       .drop_duplicates()
       .merge(df_piv2, on=["Topic", "Dp Key"], how="left")
)
df_small["Annl Edits"] = pd.to_numeric(df_small["Annl Edits"], errors="coerce").fillna(0).astype("int64")
df_small["Annl Agg Savings"] = pd.to_numeric(df_small["Annl Agg Savings"], errors="coerce").fillna(0).astype("int64")
df_small["Comments"] = "To complete topic"
df_small["DecisionDate_dt"] = pd.NaT  # for Present logic

# 4B) >10 topics: include ONLY active DPs -> Comments from Decision Status/Date logic above
df_large = ( 
    df_piv2.loc[
        df_piv2["Topic"].isin(large_topics) &
        ((df_piv2["Annl Edits"] > 0) | (df_piv2["Annl Agg Savings"] > 0))
    ]
    .merge(dec_latest[["Topic", "Dp Key", "Comments", "DecisionDate_dt"]], on=["Topic", "Dp Key"], how="left")
)
df_large["Annl Edits"] = df_large["Annl Edits"].astype("int64")
df_large["Annl Agg Savings"] = df_large["Annl Agg Savings"].astype("int64")
df_large["Comments"] = df_large["Comments"].fillna("Never presented")

df_to_present = pd.concat([df_small, df_large], ignore_index=True)

# 5) Add Policy Collection (join on DP Key)
pc_map = pd.read_excel(
    "Policy Collection Mapping.xlsx",
    engine="openpyxl",
    usecols=["DPKey", "PolicyCollection"]   # adjust names if needed
).drop_duplicates("DPKey")
pc_map["DPKey"] = pd.to_numeric(pc_map["DPKey"], errors="coerce")

df_to_present = (
    df_to_present.merge(pc_map, how="left", left_on="Dp Key", right_on="DPKey")
                 .drop(columns=["DPKey"])
                 .rename(columns={"PolicyCollection": "Policy Collection"})
)

# 6) Completing Topic (YES only for "To complete topic"; else bucket by Topic_Dp_Count in steps of 5)
def completing_topic_label(topic: str, comment: str) -> str:
    if str(comment).strip().lower() == "to complete topic":
        return "YES"
    n = topic_cnt_map.get(topic, None)
    if n is None or n <= 10:
        return "NO"
    threshold = ((int(n) - 1) // 5) * 5  # 11-15->10, 16-20->15, 21-25->20, ...
    return f"NO- Too many DPs ({threshold}+)"

df_to_present["Completing Topic"] = df_to_present.apply(
    lambda r: completing_topic_label(r["Topic"], r["Comments"]),
    axis=1
)

# 7) Present column
today = pd.Timestamp.today().normalize()
cutoff = today - pd.DateOffset(months=24)

df_to_present["Present"] = "NO"
c_lower = df_to_present["Comments"].astype(str).str.strip().str.lower()
df_to_present.loc[c_lower.isin(["never presented", "to complete topic"]), "Present"] = "YES"
df_to_present.loc[df_to_present["DecisionDate_dt"].notna() & (df_to_present["DecisionDate_dt"] <= cutoff), "Present"] = "YES"

# Final column order
df_to_present = df_to_present[
    ["Topic", "Policy Collection", "Dp Key", "Annl Edits", "Annl Agg Savings", "Present", "Comments", "Completing Topic"]
]

df_to_present.head(20)


,Topic,Policy Collection,Dp Key,Annl Edits,Annl Agg Savings,Present,Comments,Completing Topic
0,"Autologous Cultured Chondrocytes, Implant (J7330)",Age,7366,0,0,YES,To complete topic,YES
1,"Autologous Cultured Chondrocytes, Implant (J7330)",Drug requires procedure,7364,0,0,YES,To complete topic,YES
2,"Autologous Cultured Chondrocytes, Implant (J7330)",Drug requires procedure,7367,0,0,YES,To complete topic,YES
3,"Autologous Cultured Chondrocytes, Implant (J7330)",Daily maximum units,7363,0,0,YES,To complete topic,YES
4,"Autologous Cultured Chondrocytes, Implant (J7330)",Procedure requires drug,7129,23,1926,YES,To complete topic,YES
5,"Autologous Cultured Chondrocytes, Implant (J7330)",Age,7368,0,0,YES,To complete topic,YES
6,"Autologous Cultured Chondrocytes, Implant (J7330)",Modifier required,14437,0,0,YES,To complete topic,YES
7,"Autologous Cultured Chondrocytes, Implant (J7330)",All indications,7365,0,0,YES,To complete topic,YES
8,BCG (Intravesical) (J9030),Daily maximum units,11533,0,0,YES,To complete topic,YES
9,BCG (Intravesical) (J9030),All indications,9966,0,0,YES,To complete topic,YES
